<img width="8%" alt="LinkedIn.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/LinkedIn.png" style="border-radius: 15%">

# LinkedIn - Get post interactions

**Tags:** #linkedin #post #interactions #naas_drivers #content #snippet #dataframe #growth

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook fetches your profile's post interactions from a Google Sheets.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
from naas_drivers import linkedin, gsheet
import naas
import os
from datetime import date, timedelta, datetime
import pandas as pd
import naas_data_product
import math

### Setup variables
**Inputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `li_at`: Cookie used to authenticate Members and API clients.
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation.
- `limit`: Date limit.
- `force_update`: Boolean to force update.

**Outputs**
- `output_dir`: Output directory
- `file_reactions`: Name of the file with reactions to be saved in your local.
- `file_comments`: Name of the file with comments to be saved in your local.

In [ ]:
# Inputs
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET")
sheet_name = "CONTENT"
li_at = naas.secret.get("LINKEDIN_LI_AT")
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID")
limit = 20
force_update = False

# Outputs
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
file_reactions = "linkedin_post_reactions"
file_comments = "linkedin_post_comments"

## Model

### Get data from Google Sheets spreadsheet

In [ ]:
# Get data
df_posts = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
if not isinstance(df_posts, pd.DataFrame):
    df_posts = pd.DataFrame()
else:
    # Filter on this week and last week
    tw = date.today().strftime("W%W-%Y")
    lw = (date.today() - timedelta(days=date.today().weekday() + 7)).strftime("W%W-%Y")
    df_posts = df_posts[df_posts["SCENARIO"].isin([tw, lw])]

# Display result
print("Rows:", len(df_posts))
df_posts.head(1)

### Get posts interactions (reactions or comments)

In [ ]:
def get_posts_interactions(
    df_posts,
    li_at,
    JSESSIONID,
    output_dir,
    limit=20,
    force_update=False,
):
    # Init
    df_reactions = pd.DataFrame()
    df_comments = pd.DataFrame()
    if len(df_posts) == 0:
        return pd.DataFrame(), pd.DataFrame()
        
    # Loop on posts
    calls = 0
    for row in df_posts.itertuples():
        i = row.Index + 1
        title = row.TITLE
        post_url = row.CONTENT_URL
        likes_count = int(row.LIKES)
        comments_count = int(row.COMMENTS)
        activity_id = post_url.split("activity:")[1].strip()
        reaction_file = f"{activity_id}_reactions"
        comment_file = f"{activity_id}_comments"
        print(f"{i} - Starting with '{title}' ({post_url})")
        
        try:
            # Get reactions
            tmp_reactions = pload(output_dir, reaction_file)
            if (tmp_reactions is None or force_update) and likes_count > 0:
                tmp_reactions = linkedin.connect(li_at, JSESSIONID).post.get_likes(post_url, sleep=False)
                pdump(output_dir, tmp_reactions, reaction_file)
                calls += math.ceil(int(likes_count) / 100)
            r_c = 0
            if tmp_reactions is not None:
                r_c = len(tmp_reactions)
            print(f"✅ Reactions:", r_c)
            df_reactions = pd.concat([df_reactions, tmp_reactions])

            # Get comments
            tmp_comments = pload(output_dir, comment_file)
            if (tmp_comments is None or force_update) and comments_count > 0:
                tmp_comments = linkedin.connect(li_at, JSESSIONID).post.get_comments(post_url, sleep=False)
                pdump(output_dir, tmp_comments, comment_file)
                calls += math.ceil(int(comments_count) / 100)
            c_c = 0
            if tmp_comments is not None:
                c_c = len(tmp_comments)
            print(f"✅ Comments:", c_c)
            df_comments = pd.concat([df_comments, tmp_comments])
            
            # Manage limit
            print("📞 API calls:", calls)
            if calls > limit:
                break
        except Exception as e:
            print(e)
    return df_reactions, df_comments

df_reactions, df_comments = get_posts_interactions(
    df_posts,
    li_at,
    JSESSIONID,
    output_dir,
    limit,
    force_update,
)
print('👍 Total Reactions:', len(df_reactions))
print('🗨️ Total Comments:', len(df_comments))

### Add Ref Content metadata to dataframe

In [ ]:
def enrich_interactions(
    df_posts,
    df_reactions,
    df_comments
):
    # Init
    if len(df_posts) == 0:
        return pd.DataFrame(), pd.DataFrame()
    
    # Create Ref
    ref = df_posts.copy()
    ref = ref[["ENTITY", "SCENARIO", "CONTENT_URL", "TITLE", "PUBLISHED_DATE"]]

    # Merge with dfs
    df_reactions = pd.merge(ref, df_reactions, how="inner", left_on="CONTENT_URL", right_on="POST_URL")
    df_comments = pd.merge(ref, df_comments, how="inner", left_on="CONTENT_URL", right_on="POST_URL")
    return df_reactions, df_comments

df_reactions, df_comments = enrich_interactions(
    df_posts,
    df_reactions,
    df_comments
)
print('👍 Total Reactions:', len(df_reactions))
print('🗨️ Total Comments:', len(df_comments))

## Output

### Save data

In [ ]:
pdump(output_dir, df_reactions, file_reactions)
pdump(output_dir, df_comments, file_comments)